<a href="https://colab.research.google.com/github/shashank-m/covid_mining_papers/blob/master/word_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and setting up files

In [0]:
import pandas as pd
import numpy as np
import codecs
from tqdm import tqdm
import spacy
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams
import re

In [179]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [180]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)

In [181]:

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [182]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import json
import os

filenames=os.listdir("/content/drive/My Drive/covid_nlp/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv")
biorxiv="/content/drive/My Drive/covid_nlp/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv/"

## ***Preprocessing of abstract***

In [0]:
all_files=[json.load(open(biorxiv+filename, 'rb')) for filename in filenames]

In [0]:
f=all_files[32]
abstract=''
for dic in f['abstract']:
  abstract+=dic['text']
  abstract+='\n'
abstract=abstract.lower() # converts all alphabets to lower case.

In [0]:
def preprocess(abstract):
  abstract=re.sub(r'[,-?<>:!_+=#$()%^{}|~@;\'."*&[\]]',' ',abstract) #remove punctuation.
  abstract=re.sub(r'\d+',' ',abstract) # remove digits.
  abstract=re.sub(r'\s+',' ',abstract).strip() # replace multiple white spaces with single whiite space.
  return abstract

In [173]:
abstract=preprocess(abstract)
len(abstract)

5602

### **Create train and test data for CBOW model**

In [0]:
train_data=[]

In [0]:
all_tokens=re.findall(r'\S+',abstract)

In [0]:
penta_gram=ngrams(all_tokens,5)
for window in penta_gram:
  window=list(window)
  test_word=window[2]
  train_words=window[:2]+window[3:]
  train_data.append((train_words,test_word))

In [177]:

vocab=set(all_tokens)
vocab_size=len(set(all_tokens))
word_to_idx={}
for i,word in enumerate(vocab):
  word_to_idx[word]=i
vocab_size  

380

In [183]:
context_data=[]
target_data=[]
for context,target in train_data:
  inputs=torch.LongTensor([word_to_idx[word] for word in context]).view(1,-1) 
  target_vector=torch.LongTensor([word_to_idx[target]])
  context_data.append(inputs)
  target_data.append(target_vector)
len(target_data)

834

In [0]:
X_train=torch.cat(context_data,0) # put all training_data into a single tensor so that it can be batched by Dataloader.

In [0]:
X_train_label=torch.cat(target_data,0)

In [0]:
batch_size=32
training=TensorDataset(X_train,X_train_label)
train_loader=DataLoader(training,batch_size=batch_size,drop_last=True)

### CBOW model

In [187]:
class cbow(nn.Module):
  def __init__(self,vocab_size,embed_dim,context,hidden_dim,batch_size):
    super(cbow,self).__init__()
    self.bs=batch_size
    self.c=context
    self.embeddings=nn.Embedding(vocab_size,embed_dim)
    self.linear1=nn.Linear(embed_dim,hidden_dim)
    self.linear2=nn.Linear(hidden_dim,vocab_size)
    
  def forward(self,inputs):
    # if inputs.shape[0]==self.bs:
      embeds=self.embeddings(inputs)
      a_1=F.relu(self.linear1(embeds).sum(axis=1)/self.c)
      out=self.linear2(a_1)
      return out

embed_dim=300
model=cbow(vocab_size,embed_dim,4,50,batch_size)
criterion=nn.CrossEntropyLoss()
lr=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.state_dict

<bound method Module.state_dict of cbow(
  (embeddings): Embedding(380, 300)
  (linear1): Linear(in_features=300, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=380, bias=True)
)>

**Note that here training data is generated from only one sentence.We run the cbow model over 5 epochs and it looks like the model is learning something as the loss decreases every epoch. If we can overfit on this small data that means out training process works. We just need to collect more training data**

In [189]:
epochs=20
for j in range(epochs):
  for i,(x,y) in enumerate(train_loader):
    model.zero_grad()
    out=model(x)

    loss=criterion(out,y)
    loss.backward()

    optimizer.step()
  print('After epoch {},loss={}'.format(j+1,loss))    

After epoch 1,loss=5.886552810668945
After epoch 2,loss=5.637553691864014
After epoch 3,loss=5.256957054138184
After epoch 4,loss=4.88704776763916
After epoch 5,loss=4.591850757598877
After epoch 6,loss=4.321923732757568
After epoch 7,loss=4.048618316650391
After epoch 8,loss=3.7620370388031006
After epoch 9,loss=3.451620101928711
After epoch 10,loss=3.1249191761016846
After epoch 11,loss=2.7781734466552734
After epoch 12,loss=2.4235076904296875
After epoch 13,loss=2.075063467025757
After epoch 14,loss=1.7503348588943481
After epoch 15,loss=1.450291633605957
After epoch 16,loss=1.1923034191131592
After epoch 17,loss=0.9794098138809204
After epoch 18,loss=0.8130475878715515
After epoch 19,loss=0.6809931993484497
After epoch 20,loss=0.5742940902709961


**As we can see above, our tarin loss almost comes down to zero. This shows the learning process is not an issue. Time to collect more trainig data.**